In [1]:
# # 数据并行
# PyTorch非常容易的就可以使用GPU,你可以用如下方式把一个模型放到GPU上:
# device = torch.device("cuda:0")
# model.to(device)
# 然后你可以复制所有的张量到GPU上:
# mytensor = my_tensor.to(device)
# 请注意,只调用mytensor.gpu() 并没有复制张量到GPU上。你需要把它赋值给一个新的张量并在
# GPU上使用这个张量。
# 在多GPU上执行前向和反向传播是自然而然的事。然而，PyTorch默认将只是用一个GPU。你可
# 以使用DataParallel 让模型并行运行来轻易的让你的操作在多个GPU上运行。
# model = nn.DataParallel(model)

In [2]:
# 导入PyTorch模块和定义参数。
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Parameters and DataLoaders
input_size = 5
output_size = 2
batch_size = 30
data_size = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# 虚拟数据集
# 制作一个虚拟（随机）数据集，你只需实现__getitem__ 。
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
    
    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
    batch_size=batch_size, shuffle=True)

In [4]:
# 简单模型
# 作为演示，我们的模型只接受一个输入，执行一个线性操作，然后得到结果。然而，你能在任
# 何模型（CNN，RNN，Capsule Net等）上使用DataParallel 。
class Model(nn.Module):
# Our model
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
    
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
        "output size", output.size())
        return output

In [5]:
# 创建一个模型和数据并行
# 这是本教程的核心部分。首先，我们需要创建一个模型实例和检测我们是否有多个GPU。如果
# 我们有多个GPU，我们使用nn.DataParallel 来包装我们的模型。然后通过model.to(device)
# 把模型放到GPU上。
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [ ]:
# 运行模型
for data in rand_loader:
    input = data.to(device) # 数据转移到gpu上
    output = model(input)
    print("Outside: input size", input.size(),
    "output_size", output.size())
# DataParallel 自动的划分数据，并将作业发送到多个GPU上的多个模型。在每个模型完成作业
# 后， DataParallel 收集并合并结果返回给你。